# Load

In [30]:
import pandas as pd
tracks = pd.read_csv('dataset/tabular/tracks_dataset_without_outliers.csv')
# artists
artists = pd.read_csv('dataset/tabular/artists_cleaned.csv')

In [31]:
tracks

,Unnamed: 0,id,name,duration_ms,explicit,popularity,artists,album_name,danceability,energy,...,time_signature_confidence,key_confidence,mode_confidence,n_beats,n_bars,genre,year,month,day,overall_confidence
0,0,45OX2jjEw1l7lOFJfDP9fv,MONEY,168227,False,95,LISA,LALISA,0.826,0.553,...,0.989,0.043,0.253,381.0,95.0,"['k-pop', 'pop']",2021,9.0,10.0,0.51850
1,1,5XeFesFbtLpXzIVDNQP22n,I Wanna Be Yours,183956,False,94,Arctic Monkeys,AM,0.464,0.417,...,1.000,0.545,0.533,202.0,50.0,"['indie', 'garage', 'rock']",2013,9.0,9.0,0.54375
2,3,4uUG5RXrOk84mYEfFvj3cK,I'm Good (Blue),175238,True,94,David Guetta;Bebe Rexha,I'm Good (Blue),0.561,0.965,...,0.789,0.898,0.773,368.0,91.0,['pop'],2022,8.0,26.0,0.81700
3,4,0WtM2NBVQNNJLh6scP13H8,Calm Down (with Selena Gomez),239317,False,93,Rema;Selena Gomez,Calm Down (with Selena Gomez),0.801,0.806,...,1.000,0.593,0.597,424.0,105.0,['pop'],2022,8.0,25.0,0.67050
4,5,4h9wh7iOZ0GGn8QVp4RAOB,I Ain't Worried,148485,False,93,OneRepublic,I Ain’t Worried (Music From The Motion Picture...,0.704,0.797,...,1.000,0.533,0.589,339.0,83.0,"['pop', 'rock', 'piano']",2022,5.0,13.0,0.72925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80408,81220,1nSYUQngGS5foiagiilbYU,Singles You Up,182693,False,0,Jordan Davis,Mientras hago aromaterapia,0.634,0.903,...,1.000,0.439,0.533,296.0,74.0,['country'],2022,10.0,14.0,0.52725
80409,81221,1nU1tlW9ccptgRLfgq05I3,"Schlagzeile groß, Hirn zu klein",255093,False,0,Frei.Wild,Feinde deiner Feinde (Gold Edition),0.412,0.984,...,0.762,0.558,0.509,818.0,205.0,['german'],2013,4.0,19.0,0.48725
80410,81222,72ZHywsIxzF1ZkNlMyhsqt,"Ding! Dong! The Witch Is Dead (From ""The Wizar...",179035,False,0,The Countdown Kids,Happy Halloween! (Spooky Favorites for Kids),0.741,0.937,...,0.959,0.715,0.532,352.0,87.0,['children'],2022,10.0,14.0,0.75800
80411,81223,5Wt0gIt7RrXz5mJj5pslLC,Zenitsu Theme V2 (Thunder Clap and Flash!),137948,False,0,Samuel Kim,Demon Slayer: Epic Collection,0.507,0.803,...,1.000,0.791,0.703,263.0,67.0,['anime'],2021,5.0,21.0,0.75975


In [32]:
#shuffle the dataset
tracks = tracks.sample(frac=1).reset_index(drop=True)

In [33]:
tracks
# drop Unnamed:0 column
tracks = tracks.drop(columns=['Unnamed: 0'])

### merge

In [34]:
import pandas as pd
# keep only numerical columns
tracks = tracks.select_dtypes(include=['number'])

# eliminare le righe di merged_df che hanno valori nulli in qualunque colonna
tracks.dropna(inplace=True)

### scaling

In [35]:
df_temp = tracks

# df_temp = tracks_artists

In [36]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Assuming 'tracks' is a DataFrame
scaler = StandardScaler()
df_temp_scaled = scaler.fit_transform(df_temp)

# Convert back to DataFrame
df_temp_scaled = pd.DataFrame(df_temp_scaled, columns=df_temp.columns)

df = df_temp_scaled
#df = df_temp
# remove popularity column from tracks and put it in y
popularity = df.pop('popularity')
# split the data into training and testing
from sklearn.model_selection import train_test_split
#
X_train, X_test, y_train, y_test = train_test_split(df, popularity, test_size=0.2, random_state=42)
del popularity, df_temp_scaled, df_temp, artists, tracks, scaler


# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Crea il modello di base
rf = RandomForestRegressor(random_state=42)

# Definisci la griglia dei parametri
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Crea GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Fai il fit della grid search
grid_search.fit(X_train, y_train)

# Miglior modello trovato dalla grid search
best_rf = grid_search.best_estimator_

# Predici i valori sul set di test
y_pred = best_rf.predict(X_test)

# Calcola le metriche
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Best Parameters:', grid_search.best_params_)
print('Mean Squared Error:', mse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Crea il modello di base
rf = RandomForestRegressor(random_state=42)

# Definisci la griglia dei parametri
param_grid = {
    'bootstrap': [False], 'max_depth': [None], 'max_features': ['sqrt'], 'min_samples_leaf': [2], 'min_samples_split': [2], 'n_estimators': [200]
}

# Crea GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Fai il fit della grid search
grid_search.fit(X_train, y_train)

# Miglior modello trovato dalla grid search
best_rf = grid_search.best_estimator_

# Predici i valori sul set di test
y_pred = best_rf.predict(X_test)

# Calcola le metriche
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Best Parameters:', grid_search.best_params_)
print('Mean Squared Error:', mse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

In [13]:
import os
import certifi
os.environ['SSL_CERT_FILE'] = certifi.where()

In [50]:
#convert X_train, X_test, y_train, y_test to numpy
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [49]:
X_train.shape

(60403, 26)

In [37]:
#convert X_train, X_test, y_train, y_test to numpy
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

from quantile_forest import RandomForestQuantileRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

qrf = RandomForestQuantileRegressor()

qrf.fit(X_train, y_train)

y_pred = qrf.predict(X_test, quantiles=[0.5])

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Mean Squared Error:', mse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

Mean Squared Error: 0.6728011586765226
Mean Absolute Error: 0.6117925226258217
R^2: 0.3295315381191767


In [20]:
#convert X_train, X_test, y_train, y_test to numpy

from quantile_forest import RandomForestQuantileRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

qrf = RandomForestQuantileRegressor()

qrf.fit(X_train, y_train)

y_pred_quantiles = qrf.predict(X_test, quantiles=[0.025, 0.5, 0.975])

y_pred_lower = y_pred_quantiles[:, 0]  # 2.5th percentile
y_pred_median = y_pred_quantiles[:, 1]  # 50th percentile
y_pred_upper = y_pred_quantiles[:, 2]  # 97.5th percentile

mse = mean_squared_error(y_test, y_pred_lower)
mae = mean_absolute_error(y_test, y_pred_lower)
r2 = r2_score(y_test, y_pred_lower)

print('Mean Squared Error lower:', mse)
print('Mean Absolute Error lower:', mae)
print('R^2 lower:', r2)

mse = mean_squared_error(y_test, y_pred_median)
mae = mean_absolute_error(y_test, y_pred_median)
r2 = r2_score(y_test, y_pred_median)

print('Mean Squared Error median:', mse)
print('Mean Absolute Error median:', mae)
print('R^2 median:', r2)

mse = mean_squared_error(y_test, y_pred_upper)
mae = mean_absolute_error(y_test, y_pred_upper)
r2 = r2_score(y_test, y_pred_upper)

print('Mean Squared Error upper:', mse)
print('Mean Absolute Error upper:', mae)
print('R^2 upper:', r2)

Mean Squared Error lower: 4.689529609866545e-06
Mean Absolute Error lower: 9.899400614685231e-05
R^2 lower: 0.999995233241307
Mean Squared Error median: 3.0699428729538367e-06
Mean Absolute Error median: 6.376425516705464e-05
R^2 median: 0.9999968795000578
Mean Squared Error upper: 8.481484672253182e-06
Mean Absolute Error upper: 0.00016307708258974187
R^2 upper: 0.9999913788387848


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Definisci il modello con i parametri specificati
rf = RandomForestRegressor(
    bootstrap=False, 
    max_depth=None, 
    max_features='sqrt', 
    min_samples_leaf=2, 
    min_samples_split=2, 
    n_estimators=200,
    random_state=42
)

# Addestra il modello
rf.fit(X_train, y_train)

# Valuta il modello sui dati di test
y_pred = rf.predict(X_test)

# Calcola le metriche di valutazione
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R²:", r2)

# Calcola le metriche di cross-validation
cv_mse = -cross_val_score(rf, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_r2 = cross_val_score(rf, X_train, y_train, cv=5, scoring='r2')
print("Cross-Validation Mean Squared Error:", cv_mse.mean())
print("Cross-Validation Std Dev MSE:", cv_mse.std())
print("Cross-Validation R²:", cv_r2.mean())

# XGBoost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Crea il modello di base
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Definisci la griglia dei parametri
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

# Crea GridSearchCV
grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Fai il fit della grid search
grid_search.fit(X_train, y_train)

# Miglior modello trovato dalla grid search
best_xg_reg = grid_search.best_estimator_

# Predici i valori sul set di test
y_pred = best_xg_reg.predict(X_test)

# Calcola le metriche sul test set
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Best Parameters:', grid_search.best_params_)
print('Test Set Mean Squared Error:', mse)
print('Test Set Mean Absolute Error:', mae)
print('Test Set R^2:', r2)

# Cross-Validation Finale
cv_scores = cross_val_score(best_xg_reg, X_train, y_train, cv=3, scoring='neg_mean_squared_error')
cv_mse = -cv_scores.mean()
cv_std = cv_scores.std()
cv_r2_scores = cross_val_score(best_xg_reg, X_train, y_train, cv=3, scoring='r2')
cv_r2 = cv_r2_scores.mean()

print('Cross-Validation Mean Squared Error:', cv_mse)
print('Cross-Validation Std Dev MSE:', cv_std)
print('Cross-Validation R^2:', cv_r2)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Crea il modello di base
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Definisci la griglia dei parametri
# param_grid = {
#     'n_estimators': [200, 300, 400],
#     'learning_rate': [0.05, 0.1, 0.2],
#     'max_depth': [5, 7, 9],
#     'subsample': [0.8, 1.0],
#     'colsample_bytree': [0.6, 0.7, 0.8]
# }

# param_grid = {
#     'colsample_bytree': [0.6], 'learning_rate': [0.05], 'max_depth': [9], 'n_estimators': [400], 'subsample': [1.0]
#     }

param_grid = {'colsample_bytree': [0.7], 'learning_rate': [0.1], 'max_depth': [7], 'n_estimators': [200], 'subsample': [1.0]}

# Crea GridSearchCV
grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Fai il fit della grid search
grid_search.fit(X_train, y_train)

# Miglior modello trovato dalla grid search
best_xg_reg = grid_search.best_estimator_

# Predici i valori sul set di test
y_pred = best_xg_reg.predict(X_test)

# Calcola le metriche sul test set
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Best Parameters:', grid_search.best_params_)
print('Test Set Mean Squared Error:', mse)
print('Test Set Mean Absolute Error:', mae)
print('Test Set R^2:', r2)

# Cross-Validation Finale
cv_scores = cross_val_score(best_xg_reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_mse = -cv_scores.mean()
cv_std = cv_scores.std()
cv_r2_scores = cross_val_score(best_xg_reg, X_train, y_train, cv=5, scoring='r2')
cv_r2 = cv_r2_scores.mean()

print('Cross-Validation Mean Squared Error:', cv_mse)
print('Cross-Validation Std Dev MSE:', cv_std)
print('Cross-Validation R^2:', cv_r2)